# 04 — RAG Agentico

**Objetivo**: Implementar tres niveles progresivos de RAG, desde basico hasta completamente agentico con verificacion de alucinaciones.

## Contenido
1. RAG Basico: ChromaDB + retriever + generacion
2. RAG Agentico con Grading: evaluacion de relevancia de documentos
3. Agentic RAG con Hallucination Check: verificacion post-generacion
4. Evaluacion comparativa de las 3 etapas
5. Visualizacion t-SNE del espacio vectorial

In [ ]:
import os
import json
import time
import numpy as np
import matplotlib.pyplot as plt
from dotenv import load_dotenv
from openai import OpenAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pydantic import BaseModel, Field
import chromadb

load_dotenv()

llm = ChatOpenAI(model="gpt-5-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
chroma_client = chromadb.Client()

print("=" * 60)
print("RAG AGENTICO — De basico a completo")
print("=" * 60)

## Preparacion de Datos

Cargamos los comics de Batman y Spider-Man, los dividimos en chunks, y los indexamos en ChromaDB.

In [ ]:
# ============================================================
# CARGA Y CHUNKING DE DATOS
# ============================================================

def cargar_comics(ruta: str) -> list[dict]:
    """Carga comics desde un archivo JSON."""
    with open(ruta) as f:
        return json.load(f)

batman_comics = cargar_comics("../data/batman_comics.json")
spider_comics = cargar_comics("../data/spiderman_comics.json")

print(f"Batman comics: {len(batman_comics)}")
print(f"Spider-Man comics: {len(spider_comics)}")

# Chunking
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    separators=["\n\n", "\n", ". ", " ", ""]
)

chunks = []
metadatas = []

for comic in batman_comics + spider_comics:
    splits = splitter.split_text(comic["contenido"])
    for i, chunk in enumerate(splits):
        chunks.append(chunk)
        metadatas.append({
            "personaje": comic["personaje"],
            "arco": comic["arco"],
            "tema": comic["tema"],
            "titulo": comic["titulo"],
            "doc_id": comic["id"],
            "chunk_id": i,
        })

print(f"\nTotal chunks: {len(chunks)}")
print(f"Chunks Batman: {sum(1 for m in metadatas if m['personaje'] == 'batman')}")
print(f"Chunks Spider-Man: {sum(1 for m in metadatas if m['personaje'] == 'spiderman')}")
print(f"\nEjemplo de chunk:")
print(f"  Texto: {chunks[0][:150]}...")
print(f"  Metadata: {metadatas[0]}")

In [ ]:
# ============================================================
# INDEXACION EN CHROMADB
# ============================================================

# Generar embeddings en batch
print("Generando embeddings...")
t0 = time.time()
all_embeddings = embeddings.embed_documents(chunks)
embed_time = time.time() - t0
print(f"  {len(all_embeddings)} embeddings generados en {embed_time:.1f}s")
print(f"  Dimension: {len(all_embeddings[0])}")

# Crear coleccion en ChromaDB
collection = chroma_client.create_collection(
    name="comics_rag",
    metadata={"hnsw:space": "cosine"}
)

# Insertar chunks
collection.add(
    ids=[f"chunk_{i}" for i in range(len(chunks))],
    embeddings=all_embeddings,
    documents=chunks,
    metadatas=metadatas,
)

print(f"\nColeccion 'comics_rag' creada con {collection.count()} documentos")

## Etapa 1: RAG Basico

Retriever simple → Contexto → Generacion. Sin evaluacion de calidad.

```
Query → [Retriever (k=4)] → [Contexto] → [LLM] → Respuesta
```

In [ ]:
# ============================================================
# ETAPA 1: RAG BASICO
# ============================================================

def rag_basico(query: str, k: int = 4) -> dict:
    """
    RAG basico: retrieve + generate.
    
    Args:
        query: Pregunta del usuario.
        k: Numero de documentos a recuperar.
    
    Returns:
        Dict con respuesta, documentos, y metricas.
    """
    t0 = time.time()
    
    # Retrieve
    query_embedding = embeddings.embed_query(query)
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )
    
    docs = results["documents"][0]
    metas = results["metadatas"][0]
    dists = results["distances"][0]
    
    # Build context
    contexto = "\n\n".join([
        f"[{m['titulo']} - {m['arco']}]: {doc}"
        for doc, m in zip(docs, metas)
    ])
    
    # Generate
    response = llm.invoke([
        SystemMessage(content=f"""Eres un experto en comics. Responde SOLO con informacion del contexto.
Si no hay informacion suficiente, di "No tengo informacion suficiente."
Responde en español.

Contexto:
{contexto}"""),
        HumanMessage(content=query),
    ])
    
    latencia = (time.time() - t0) * 1000
    
    return {
        "query": query,
        "respuesta": response.content,
        "docs_recuperados": [{"texto": d[:100], "meta": m, "distancia": dist} for d, m, dist in zip(docs, metas, dists)],
        "num_docs": len(docs),
        "latencia_ms": round(latencia, 1),
        "etapa": "basico",
    }


# Test
r1 = rag_basico("Como se convirtio Bruce Wayne en Batman?")
print("=" * 60)
print("ETAPA 1: RAG BASICO")
print("=" * 60)
print(f"\nQuery: {r1['query']}")
print(f"Docs recuperados: {r1['num_docs']}")
for i, doc in enumerate(r1["docs_recuperados"]):
    print(f"  [{i+1}] {doc['meta']['personaje']}/{doc['meta']['arco']} (dist={doc['distancia']:.3f})")
print(f"\nRespuesta: {r1['respuesta'][:400]}...")
print(f"Latencia: {r1['latencia_ms']}ms")

## Etapa 2: RAG Agentico con Grading

Agregamos un nodo "grader" que evalua la relevancia de cada documento recuperado.
Si menos de 2 documentos son relevantes, reescribimos la query y reintentamos.

```
Query → [Retrieve] → [Grade docs] → ¿>=2 relevantes?
                                      ├── SI → [Generate]
                                      └── NO → [Rewrite query] → [Retrieve] → ...
```

In [ ]:
# ============================================================
# ETAPA 2: GRADING DE RELEVANCIA
# ============================================================

from typing import Literal, TypedDict, Annotated
from langgraph.graph import StateGraph, START, END

class GradeDocument(BaseModel):
    """Evaluacion de relevancia de un documento."""
    es_relevante: Literal["si", "no"] = Field(
        description="El documento es relevante para responder la pregunta"
    )
    razon: str = Field(description="Breve razon de la decision")

grader_llm = llm.with_structured_output(GradeDocument)


def evaluar_relevancia(query: str, documento: str) -> GradeDocument:
    """Evalua si un documento es relevante para una query."""
    return grader_llm.invoke(
        f"Query del usuario: {query}\n\nDocumento recuperado:\n{documento}\n\n"
        f"¿Este documento contiene informacion relevante para responder la query?"
    )


class RAGState(TypedDict):
    query: str
    query_original: str
    documentos: list[dict]
    docs_relevantes: list[dict]
    respuesta: str
    intentos: int
    etapa: str
    metricas: dict


def nodo_retrieve(state: RAGState) -> dict:
    """Recupera documentos de ChromaDB."""
    query_emb = embeddings.embed_query(state["query"])
    results = collection.query(
        query_embeddings=[query_emb], n_results=4,
        include=["documents", "metadatas", "distances"]
    )
    docs = [
        {"texto": d, "meta": m, "distancia": dist}
        for d, m, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0])
    ]
    return {"documentos": docs}


def nodo_grade(state: RAGState) -> dict:
    """Evalua relevancia de cada documento."""
    relevantes = []
    for doc in state["documentos"]:
        grade = evaluar_relevancia(state["query"], doc["texto"])
        if grade.es_relevante == "si":
            relevantes.append(doc)
    return {"docs_relevantes": relevantes}


def decidir_generacion(state: RAGState) -> str:
    """Decide si generar o reescribir."""
    if len(state.get("docs_relevantes", [])) >= 2:
        return "generar"
    if state.get("intentos", 0) >= 2:
        return "generar"  # Generar con lo que hay
    return "reescribir"


def nodo_reescribir(state: RAGState) -> dict:
    """Reescribe la query para mejorar la recuperacion."""
    response = llm.invoke([
        SystemMessage(content="Reescribe la siguiente pregunta para mejorar la busqueda en una base de datos de comics. Manten el significado pero usa diferentes palabras clave."),
        HumanMessage(content=state["query"]),
    ])
    return {"query": response.content, "intentos": state.get("intentos", 0) + 1}


def nodo_generar(state: RAGState) -> dict:
    """Genera respuesta con los documentos relevantes."""
    docs = state.get("docs_relevantes", []) or state.get("documentos", [])
    contexto = "\n\n".join([d["texto"] for d in docs[:4]])
    
    response = llm.invoke([
        SystemMessage(content=f"Responde basandote SOLO en el contexto. En español.\n\nContexto:\n{contexto}"),
        HumanMessage(content=state["query_original"]),
    ])
    return {"respuesta": response.content, "etapa": "grading"}


# Build graph
graph = StateGraph(RAGState)
graph.add_node("retrieve", nodo_retrieve)
graph.add_node("grade", nodo_grade)
graph.add_node("reescribir", nodo_reescribir)
graph.add_node("generar", nodo_generar)

graph.add_edge(START, "retrieve")
graph.add_edge("retrieve", "grade")
graph.add_conditional_edges("grade", decidir_generacion, {
    "generar": "generar",
    "reescribir": "reescribir",
})
graph.add_edge("reescribir", "retrieve")
graph.add_edge("generar", END)

rag_grading_app = graph.compile()
print("Grafo RAG con grading compilado")

In [ ]:
# Test
t0 = time.time()
r2 = rag_grading_app.invoke({
    "query": "Como se convirtio Bruce Wayne en Batman?",
    "query_original": "Como se convirtio Bruce Wayne en Batman?",
    "documentos": [],
    "docs_relevantes": [],
    "respuesta": "",
    "intentos": 0,
    "etapa": "",
    "metricas": {},
})
lat2 = (time.time() - t0) * 1000

print("=" * 60)
print("ETAPA 2: RAG CON GRADING")
print("=" * 60)
print(f"Docs recuperados: {len(r2['documentos'])}")
print(f"Docs relevantes: {len(r2['docs_relevantes'])}")
print(f"Intentos de reescritura: {r2['intentos']}")
print(f"Respuesta: {r2['respuesta'][:400]}...")
print(f"Latencia: {lat2:.0f}ms")

## Etapa 3: Agentic RAG con Hallucination Check

Agregamos verificacion post-generacion: el LLM evalua si la respuesta esta fundamentada en el contexto.

```
Query → [Retrieve] → [Grade] → [Decide] → [Generate] → [Hallucination Check] → [Decide Output]
                                    │                           │
                                    └── [Rewrite] ◄─────────────┘ (si alucina)
```

In [ ]:
# ============================================================
# ETAPA 3: AGENTIC RAG CON HALLUCINATION CHECK
# ============================================================

class HallucinationCheck(BaseModel):
    """Evaluacion de alucinacion."""
    esta_fundamentada: Literal["si", "no"] = Field(
        description="La respuesta esta fundamentada en el contexto"
    )
    score: int = Field(description="Score de grounding 1-5", ge=1, le=5)
    detalle: str = Field(description="Que partes estan o no fundamentadas")

hallucination_llm = llm.with_structured_output(HallucinationCheck)


class FullRAGState(TypedDict):
    query: str
    query_original: str
    documentos: list[dict]
    docs_relevantes: list[dict]
    respuesta: str
    hallucination_check: dict | None
    intentos: int
    intentos_hallucination: int
    etapa: str


def nodo_retrieve_full(state: FullRAGState) -> dict:
    query_emb = embeddings.embed_query(state["query"])
    results = collection.query(
        query_embeddings=[query_emb], n_results=4,
        include=["documents", "metadatas", "distances"]
    )
    docs = [
        {"texto": d, "meta": m, "distancia": dist}
        for d, m, dist in zip(results["documents"][0], results["metadatas"][0], results["distances"][0])
    ]
    return {"documentos": docs}


def nodo_grade_full(state: FullRAGState) -> dict:
    relevantes = []
    for doc in state["documentos"]:
        grade = evaluar_relevancia(state["query"], doc["texto"])
        if grade.es_relevante == "si":
            relevantes.append(doc)
    return {"docs_relevantes": relevantes}


def decidir_gen_full(state: FullRAGState) -> str:
    if len(state.get("docs_relevantes", [])) >= 2:
        return "generar"
    if state.get("intentos", 0) >= 2:
        return "generar"
    return "reescribir"


def nodo_reescribir_full(state: FullRAGState) -> dict:
    response = llm.invoke([
        SystemMessage(content="Reescribe esta pregunta con diferentes palabras clave para buscar en una base de comics."),
        HumanMessage(content=state["query"]),
    ])
    return {"query": response.content, "intentos": state.get("intentos", 0) + 1}


def nodo_generar_full(state: FullRAGState) -> dict:
    docs = state.get("docs_relevantes", []) or state.get("documentos", [])
    contexto = "\n\n".join([d["texto"] for d in docs[:4]])
    response = llm.invoke([
        SystemMessage(content=f"Responde SOLO con informacion del contexto. En español.\n\nContexto:\n{contexto}"),
        HumanMessage(content=state["query_original"]),
    ])
    return {"respuesta": response.content}


def nodo_hallucination_check(state: FullRAGState) -> dict:
    docs = state.get("docs_relevantes", []) or state.get("documentos", [])
    contexto = "\n\n".join([d["texto"] for d in docs[:4]])
    check = hallucination_llm.invoke(
        f"Contexto disponible:\n{contexto}\n\nRespuesta generada:\n{state['respuesta']}\n\n"
        f"¿La respuesta esta completamente fundamentada en el contexto?"
    )
    return {"hallucination_check": check.model_dump(), "intentos_hallucination": state.get("intentos_hallucination", 0) + 1}


def decidir_output(state: FullRAGState) -> str:
    check = state.get("hallucination_check", {})
    if check.get("esta_fundamentada") == "si":
        return "output"
    if state.get("intentos_hallucination", 0) >= 2:
        return "output"  # Output with warning
    return "regenerar"


# Build full graph
full_graph = StateGraph(FullRAGState)
full_graph.add_node("retrieve", nodo_retrieve_full)
full_graph.add_node("grade", nodo_grade_full)
full_graph.add_node("reescribir", nodo_reescribir_full)
full_graph.add_node("generar", nodo_generar_full)
full_graph.add_node("hallucination_check", nodo_hallucination_check)

full_graph.add_edge(START, "retrieve")
full_graph.add_edge("retrieve", "grade")
full_graph.add_conditional_edges("grade", decidir_gen_full, {
    "generar": "generar",
    "reescribir": "reescribir",
})
full_graph.add_edge("reescribir", "retrieve")
full_graph.add_edge("generar", "hallucination_check")
full_graph.add_conditional_edges("hallucination_check", decidir_output, {
    "output": END,
    "regenerar": "generar",
})

rag_full_app = full_graph.compile()
print("Grafo RAG completo (con hallucination check) compilado")

In [ ]:
t0 = time.time()
r3 = rag_full_app.invoke({
    "query": "Como se convirtio Bruce Wayne en Batman?",
    "query_original": "Como se convirtio Bruce Wayne en Batman?",
    "documentos": [],
    "docs_relevantes": [],
    "respuesta": "",
    "hallucination_check": None,
    "intentos": 0,
    "intentos_hallucination": 0,
    "etapa": "full",
})
lat3 = (time.time() - t0) * 1000

print("=" * 60)
print("ETAPA 3: AGENTIC RAG COMPLETO")
print("=" * 60)
print(f"Docs relevantes: {len(r3['docs_relevantes'])}")
print(f"Hallucination check: {r3['hallucination_check']}")
print(f"Respuesta: {r3['respuesta'][:400]}...")
print(f"Latencia: {lat3:.0f}ms")

## 4. Evaluacion Comparativa

Ejecutamos las 3 etapas con las preguntas de evaluacion y comparamos.

In [ ]:
# ============================================================
# EVALUACION COMPARATIVA
# ============================================================

# Cargar preguntas de evaluacion
eval_questions = []
with open("../data/comics_eval.jsonl") as f:
    for line in f:
        eval_questions.append(json.loads(line))

# Evaluar solo las primeras 5 para no gastar mucho
eval_subset = eval_questions[:5]

resultados_eval = []

print("=" * 60)
print("EVALUACION COMPARATIVA (5 preguntas)")
print("=" * 60)

for eq in eval_subset:
    pregunta = eq["pregunta"]
    keywords = eq["keywords"]
    
    # Etapa 1: RAG Basico
    t0 = time.time()
    r_basico = rag_basico(pregunta)
    lat_basico = (time.time() - t0) * 1000
    
    # Keyword recall
    resp_lower = r_basico["respuesta"].lower()
    kw_found = sum(1 for kw in keywords if kw.lower() in resp_lower)
    recall_basico = kw_found / len(keywords)
    
    # Etapa 2: RAG con Grading
    t0 = time.time()
    r_grading = rag_grading_app.invoke({
        "query": pregunta, "query_original": pregunta,
        "documentos": [], "docs_relevantes": [],
        "respuesta": "", "intentos": 0, "etapa": "", "metricas": {},
    })
    lat_grading = (time.time() - t0) * 1000
    
    resp_lower = r_grading["respuesta"].lower()
    kw_found = sum(1 for kw in keywords if kw.lower() in resp_lower)
    recall_grading = kw_found / len(keywords)
    
    # Etapa 3: Full Agentic
    t0 = time.time()
    r_full = rag_full_app.invoke({
        "query": pregunta, "query_original": pregunta,
        "documentos": [], "docs_relevantes": [],
        "respuesta": "", "hallucination_check": None,
        "intentos": 0, "intentos_hallucination": 0, "etapa": "full",
    })
    lat_full = (time.time() - t0) * 1000
    
    resp_lower = r_full["respuesta"].lower()
    kw_found = sum(1 for kw in keywords if kw.lower() in resp_lower)
    recall_full = kw_found / len(keywords)
    
    resultados_eval.append({
        "pregunta": pregunta[:40],
        "recall_basico": recall_basico,
        "recall_grading": recall_grading,
        "recall_full": recall_full,
        "lat_basico": lat_basico,
        "lat_grading": lat_grading,
        "lat_full": lat_full,
    })
    
    print(f"\n  Q: {pregunta[:50]}...")
    print(f"    Basico:  recall={recall_basico:.0%} lat={lat_basico:.0f}ms")
    print(f"    Grading: recall={recall_grading:.0%} lat={lat_grading:.0f}ms")
    print(f"    Full:    recall={recall_full:.0%} lat={lat_full:.0f}ms")

In [ ]:
# ============================================================
# VISUALIZACION COMPARATIVA
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Panel 1: Keyword Recall por etapa
preguntas_labels = [r["pregunta"][:25] + "..." for r in resultados_eval]
x = np.arange(len(preguntas_labels))
width = 0.25

bars1 = axes[0].bar(x - width, [r["recall_basico"] for r in resultados_eval], width, label="Basico", color="#FF5722", alpha=0.8)
bars2 = axes[0].bar(x, [r["recall_grading"] for r in resultados_eval], width, label="Grading", color="#2196F3", alpha=0.8)
bars3 = axes[0].bar(x + width, [r["recall_full"] for r in resultados_eval], width, label="Full Agentic", color="#4CAF50", alpha=0.8)

axes[0].set_ylabel("Keyword Recall")
axes[0].set_title("Calidad: Keyword Recall por Etapa")
axes[0].set_xticks(x)
axes[0].set_xticklabels(preguntas_labels, rotation=45, ha="right", fontsize=7)
axes[0].legend()
axes[0].set_ylim(0, 1.1)

# Panel 2: Latencia por etapa
axes[1].bar(x - width, [r["lat_basico"] for r in resultados_eval], width, label="Basico", color="#FF5722", alpha=0.8)
axes[1].bar(x, [r["lat_grading"] for r in resultados_eval], width, label="Grading", color="#2196F3", alpha=0.8)
axes[1].bar(x + width, [r["lat_full"] for r in resultados_eval], width, label="Full Agentic", color="#4CAF50", alpha=0.8)

axes[1].set_ylabel("Latencia (ms)")
axes[1].set_title("Costo: Latencia por Etapa")
axes[1].set_xticks(x)
axes[1].set_xticklabels(preguntas_labels, rotation=45, ha="right", fontsize=7)
axes[1].legend()

plt.tight_layout()
plt.savefig("../data/rag_comparacion.png", dpi=150, bbox_inches="tight")
plt.show()

## 5. Visualizacion t-SNE del Espacio Vectorial

In [ ]:
# ============================================================
# VISUALIZACION t-SNE
# ============================================================

from sklearn.manifold import TSNE

# Obtener todos los embeddings y metadatas de ChromaDB
all_data = collection.get(include=["embeddings", "metadatas"])
embs = np.array(all_data["embeddings"])
metas = all_data["metadatas"]

# t-SNE reduction
tsne = TSNE(n_components=2, random_state=42, perplexity=min(30, len(embs) - 1))
embs_2d = tsne.fit_transform(embs)

# Plot
fig, ax = plt.subplots(figsize=(12, 8))

colores = {"batman": "#1a1a2e", "spiderman": "#e63946"}
marcadores = {"batman": "o", "spiderman": "s"}

for i, (x, y) in enumerate(embs_2d):
    personaje = metas[i]["personaje"]
    ax.scatter(x, y, c=colores[personaje], marker=marcadores[personaje], s=50, alpha=0.7)

# Query embeddings
query_texts = ["Batman origen", "Spider-Man poderes", "Comparar heroes"]
query_embs = [embeddings.embed_query(q) for q in query_texts]
query_all = np.vstack([embs, query_embs])
tsne_all = TSNE(n_components=2, random_state=42, perplexity=min(30, len(query_all) - 1))
all_2d = tsne_all.fit_transform(query_all)

# Re-plot with queries
fig, ax = plt.subplots(figsize=(12, 8))

for i in range(len(embs)):
    personaje = metas[i]["personaje"]
    ax.scatter(all_2d[i, 0], all_2d[i, 1], c=colores[personaje], 
               marker=marcadores[personaje], s=40, alpha=0.5)

# Plot queries
for i, q_text in enumerate(query_texts):
    idx = len(embs) + i
    ax.scatter(all_2d[idx, 0], all_2d[idx, 1], c="gold", marker="*", s=200, 
               edgecolors="black", linewidth=1.5, zorder=5)
    ax.annotate(q_text, (all_2d[idx, 0], all_2d[idx, 1]), 
                textcoords="offset points", xytext=(10, 10), fontsize=9, fontweight="bold")

from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker="o", color="w", markerfacecolor="#1a1a2e", markersize=10, label="Batman"),
    Line2D([0], [0], marker="s", color="w", markerfacecolor="#e63946", markersize=10, label="Spider-Man"),
    Line2D([0], [0], marker="*", color="w", markerfacecolor="gold", markersize=15, markeredgecolor="black", label="Query"),
]
ax.legend(handles=legend_elements, loc="upper right")
ax.set_title("Espacio Vectorial t-SNE: Comics Batman vs Spider-Man", fontsize=14)
ax.set_xlabel("t-SNE dim 1")
ax.set_ylabel("t-SNE dim 2")

plt.tight_layout()
plt.savefig("../data/tsne_comics.png", dpi=150, bbox_inches="tight")
plt.show()

print(f"Total puntos: {len(embs)} chunks + {len(query_texts)} queries")

## Takeaways

1. **RAG Basico** es rapido pero puede recuperar documentos irrelevantes
2. **RAG con Grading** mejora la precision descartando docs irrelevantes, a costa de latencia extra
3. **Agentic RAG completo** agrega verificacion de alucinaciones, maximizando calidad
4. El trade-off es siempre **calidad vs costo/latencia** — elegir segun el caso de uso
5. t-SNE muestra que los embeddings separan bien los universos Batman vs Spider-Man
6. Las queries se ubican cerca de los clusters relevantes en el espacio vectorial